## Importing packages

In [535]:
# setup
import numpy as np

In [536]:
import sys
print(sys.version)

2.7.16 (default, Apr  6 2019, 01:42:57) 
[GCC 8.3.0]


In [537]:
import sklearn
from sklearn import linear_model as lm
sklearn.__version__

'0.20.4'

## Importing data

In [539]:
# svm.py
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv 
#import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score
from sklearn.utils import shuffle

In [540]:
# Toy data
import pandas as pd


X_train= pd.read_csv('Xtr_mat100.csv',header=None)
X_test = pd.read_csv('Xte_mat100.csv', header=None)


In [639]:
Y_train = pd.read_csv('Ytr.csv')
y = Y_train['Bound'].to_numpy()
y=2*y-1
y

array([ 1, -1,  1, ...,  1,  1,  1])

## Data preprocessing

In [542]:
def string_to_float(data):
    X=[]
    n = data.shape[0]
    k=10
    for i in data.index:
        lis = data.iloc[i, :].str.split()
        newlis = np.array(lis.tolist(),dtype=float)
        newlis = newlis.reshape(1,-1)
        X.append(newlis)
   
    return X    

In [543]:
Xtr=string_to_float(X_train)
Xte=string_to_float(X_test)
New_Xtr = np.array(Xtr).squeeze()
New_Xte = np.array(Xte).squeeze()

## Models implementation

#### Linear regression

In [544]:
# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)

    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam * n
    b = X.T.dot(y)

    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta) 



In [545]:
# Weighted Ridge Regression (WRR)
def solveWRR(y, X, w, lam):
    n, p = X.shape
    assert (len(y) == len(w) == n)

    y1 = np.sqrt(w) * y
    X1 = (np.sqrt(w) * X.T).T
    # Hint:
    # Find y1 and X1 such that:
    beta = solveRR(y1, X1, lam)
    return (beta) 

In [546]:
# Logistic Ridge Regression (LRR)
def solveLRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    #Parameters
    max_iter=100
    eps=1e-3
    sigmoid = lambda a: 1/(1 + np.exp(-a))
    
    #Initialize
    beta = np.zeros(p)
    
            
    # Hint: Use IRLS
    for i in range(max_iter):
        beta_old = beta
        f= X.dot(beta_old)
        W=sigmoid(f)*sigmoid(-f)
        z=f.T+y/sigmoid(y*f)
        beta = solveWRR(z, X, W, 2*lam)
    # Break condition
        if np.sum((beta-beta_old)**2) < eps:
            break
    return (beta)

#### Splitting data

In [739]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test =  train_test_split(Data_tr, y,test_size=0.7,random_state=42)

In [740]:
# Fit our model and compute its parameters
lam = 0.1
beta = solveLRR(Y_train, X_train, lam)

In [741]:
# Compute predicted probabilities and classes
probas_pred = 1/(1+np.exp(-beta.T.dot(X_test.T)))
y_pred = np.round(probas_pred)

In [742]:
from sklearn.metrics import accuracy_score, roc_auc_score

print("Our model's performance:")
print('Accuracy: {:.2%}'.format(accuracy_score(Y_test, y_pred)))
print('AUC: {:.2%}'.format(roc_auc_score(Y_test, probas_pred)))

Our model's performance:
Accuracy: 49.57%
AUC: 49.17%


## SVM implementation

In [567]:
data = pd.DataFrame(New_Xtr)

In [568]:
data_normalized = MinMaxScaler().fit_transform(New_Xtr)
data = pd.DataFrame(data_normalized)

In [569]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.090909,0.125,0.142857,0.428571,0.166667,0.375,0.166667,0.000000,0.000000,0.166667,...,0.181818,0.000,0.0,0.000000,0.2,0.000000,0.000000,0.117647,0.000000,0.000000
1,0.000000,0.125,0.285714,0.000000,0.333333,0.250,0.000000,0.166667,0.333333,0.166667,...,0.272727,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.294118,0.000000,0.166667
2,0.181818,0.000,0.000000,0.000000,0.666667,0.125,0.166667,0.166667,0.000000,0.333333,...,0.000000,0.125,0.2,0.000000,0.0,0.090909,0.000000,0.117647,0.000000,0.166667
3,0.000000,0.250,0.428571,0.000000,0.333333,0.000,0.500000,0.166667,0.333333,0.000000,...,0.090909,0.125,0.0,0.000000,0.0,0.000000,0.000000,0.058824,0.142857,0.333333
4,0.090909,0.500,0.000000,0.428571,0.166667,0.125,0.000000,0.166667,0.166667,0.000000,...,0.000000,0.000,0.2,0.272727,0.4,0.272727,0.142857,0.058824,0.000000,0.000000


In [570]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = reg_strength * (np.sum(distances) / N)
    
    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [571]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])
    d = 1 - (Y_batch * np.dot(X_batch, W))
    dw = W if max(0, d) == 0 else (W - (reg_strength * Y_batch * X_batch))
    

    return dw

In [572]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    # stochastic gradient descent
    for epoch in range(1, max_epochs): 
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
            
    return weights

In [727]:
def sgd1(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        print('After shffle', X)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [745]:
def init():
   
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(dtrain, y, test_size=0.5, random_state=42)
    

    # train the model
    print("training started...")
    W = sgd1(X_train, y_train)
    print("training finished.")
    print("weights are: {}".format(W))
    
    
    # testing the model on test set
    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(W, X_test[i])) #model
        y_test_predicted = np.append(y_test_predicted, yp)
    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))

In [746]:
#set hyper-parameters and call init
#hyper-parameters are normally tuned using cross-validation
#but following work good enough
reg_strength = 10000 # regularization strength
learning_rate = 0.001
init()

splitting dataset into train and test sets...
training started...
('After shffle', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))
Epoch is:1 and Cost is: 5180.0
('After shffle', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))
Epoch is:2 and Cost is: 5180.0
training finished.
weights are: [0. 0. 0. ... 0. 0. 0.]
accuracy on test dataset: 0.514
recall on test dataset: 0.994186046512
precision on test dataset: 0.994186046512


## Kernel implementation

#### Data preprocessing with spectrum kernel and one hot encoder

In [577]:
df_tr = pd.read_csv('Xtr.csv')
df_te = pd.read_csv('Xte.csv')

In [578]:
import pandas as pd
kmer_size = 3

def base2int(c):
    return {'A': 0, 'C':1, 'G':2, 'T':3}.get(c,0)

def get_kmers(sequence, kmer_size=3):
    return [sequence[i: i + kmer_size] for i in range(len(sequence) - kmer_size)]

In [579]:
def base2int(c):
    return {'A': 0, 'C': 1, 'G': 2, 'T': 3}.get(c,0)

def index(kmer):
    #Transform the kmer into sequence of character indices
    base_indices = np.array([base2int(base) for base in kmer])
    multiplier = 4**np.arange(len(kmer))  #[4**0, 4**1, 4**2, ...]
    kmer_index = multiplier.dot(base_indices)
    
    return kmer_index

In [580]:
def spectrum_embedding1(sequence):
    kmers = get_kmers(sequence)
    kmer_indices = [index(kmer) for kmer in kmers]
    one_hot_vector = np.zeros(4**kmer_size)
    for kmer_index in kmer_indices:
        one_hot_vector[kmer_index] +=1
    return one_hot_vector 

In [581]:
from sklearn.preprocessing import OneHotEncoder


def spectrum_embedding(data,k, train = True):
    out = []
    for i in range(len(data)):
        line = data.iloc[i]['seq']
        kmers = get_kmers(line,k)
        out.append(kmers)
        
    data = pd.DataFrame(data=out,columns=['txt'+str(i) for i in range(len(kmers))])
    
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(data)
    return data,encoded_data


In [766]:
from sklearn.preprocessing import OneHotEncoder


def spectrum_embedding(data, data_test, k):
    out, out2 = [], []
    for i in range(len(data)):
        line = data.iloc[i]['seq']
        kmers = get_kmers(line,k)
        out.append(kmers)
        
    for i in range(len(data_test)):
        line = data_test.iloc[i]['seq']
        kmers = get_kmers(line,k)
        out2.append(kmers)
        
    data = pd.DataFrame(data=out,columns=['txt'+str(i) for i in range(len(kmers))])
    data_test = pd.DataFrame(data=out2,columns=['txt'+str(i) for i in range(len(kmers))])
    
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_data = encoder.fit(data)
        
    data_train = encoded_data.transform(data)
    data_test = encoded_data.transform(data_test)
    return data_train, data_test


In [767]:
dtrain, dtest = spectrum_embedding(df_tr, df_te, k=5)

In [584]:
Data_tr = [spectrum_embedding1(i) for i in df_tr['seq']]
Data_te = [spectrum_embedding1(j) for j in df_te['seq']]

In [585]:
Data_tr = pd.DataFrame(Data_tr) 
Data_te = pd.DataFrame(Data_te) 

In [586]:
Data_tr = Data_tr.reset_index()
Data_te = Data_te.reset_index()

In [587]:
Data_tr = Data_tr.rename(columns={'index':'Id'})
Data_te = Data_te.rename(columns={'index':'Id'})

In [588]:
Data_tr = Data_tr.to_numpy()
Data_te = Data_te.to_numpy()

In [658]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(Data_tr, y, test_size = 0.5, random_state=101)

#### Models Implementation 

#### Kernel ridge

In [659]:
import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

In [660]:
def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K

In [661]:
def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm + X2_norm - 2 * np.dot(X1, X2.T)))
    return K

In [662]:
def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

#print(sigma_from_median(New_Xtr))

In [717]:
class KernelRidge():
    '''
    Kernel Ridge Regression
    
    Methods
    ----
    fit
    predict
    '''
    def __init__(self, sigma=None, lambd=0.1):
        self.kernel = rbf_kernel
        self.sigma = sigma
        self.lambd = lambd

    def fit(self, X, y):
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        
        # Compute default sigma from data
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
        
        A = self.kernel(X, X, sigma=self.sigma) + n * self.lambd * np.eye(n)
        
        ## self.alpha = (K + n lambda I)^-1 y
        # Solution to A x = y
        self.alpha = np.linalg.solve(A , y)

        return self
        
    def predict(self, X):
        # Prediction rule: 
        K_x = self.kernel(X, self.X_train, sigma=self.sigma)
        return K_x.dot(self.alpha)

In [719]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(New_Xtr, y, test_size = 0.5, random_state=101)

In [726]:
from sklearn.metrics import accuracy_score
model = KernelRidge(lambd=0.1, sigma=None)
y_pred = model.fit(X_train, y_train).predict(X_test)

#### Kernel Ridge regression

In [601]:
def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the RBF kernel with parameter sigma
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    sigma: float
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

In [602]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    
    Methods
    ----
    fit
    predict
    '''
    kernels_ = {
        'linear': linear_kernel,
        'polynomial': polynomial_kernel,
        'rbf': rbf_kernel,
        # 'custom_kernel': custom_kernel, # Your kernel
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        
    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf':
            params['sigma'] = kwargs.get('sigma', 1.)
        if self.kernel_name == 'polynomial':
            params['degree'] = kwargs.get('degree', 2)
        # if self.kernel_name == 'custom_kernel':
        #     params['parameter_1'] = kwargs.get('parameter_1', None)
        #     params['parameter_2'] = kwargs.get('parameter_2', None)
        return params

    def fit(self, X, y, **kwargs):
        return self
        
    def decision_function(self, X):
        pass

    def predict(self, X):
        pass

In [603]:
def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def polynomial_kernel(X1, X2, degree=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the polynomial kernel of degree `degree`
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    degree: int
    '''
    return (1 + linear_kernel(X1, X2))**degree

In [604]:
class KernelRidgeRegression(KernelMethodBase):
    '''
    Kernel Ridge Regression
    '''
    def __init__(self, lambd=0.1, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelRidgeRegression, self).__init__(**kwargs)

    def fit(self, X, y):

        self.X_train = X
        self.y_train = y
        n = len(self.y_train)
        
        A = self.kernel_function_(X, X, **self.kernel_parameters)
        A[np.diag_indices_from(A)] += self.lambd * n
        # self.alpha = (K + n lambda I)^-1 y
        self.alpha = np.linalg.solve(A , self.y_train)

        return self
    
    
    def decision_function(self, X):
        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)
        return np.dot(K_x,self.alpha)
    
    def predict(self, X):
        proba = self.decision_function(X)
        predicted_classes = np.where(proba <=0.5, 0, 1)
        #pred= np.where(predicted_classes==-1,0,1)
        return predicted_classes
    
    def error(self,ypred, ytrue):
        e = (ypred == ytrue).mean()
        return e
    
    
class WeightedKernelRidgeRegression(KernelRidgeRegression):
    '''
    Weighted Kernel Ridge Regression
    
    This is just used for the KernelLogistic following up
    '''
    def fit(self, K, y, sample_weights=None):

        self.y_train = y
        n = len(self.y_train)
        
        w = np.ones_like(self.y_train) if sample_weights is None else sample_weights
        W = np.diag(np.sqrt(w))
        
        A = W.dot(K).dot(W)
        A[np.diag_indices_from(A)] += self.lambd * n
        # self.alpha = W (K + n lambda I)^-1 W y
        self.alpha = W.dot(np.linalg.solve(A , W.dot(self.y_train)))

        return self

In [692]:
class KernelRidgeClassifier(KernelRidgeRegression):
    '''
    Kernel Ridge Classification
    '''
    def predict(self, X):
        return np.sign(self.decision_function(X))

In [714]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(dtrain, y, test_size = 0.5, random_state=101)

In [716]:
kernel = 'rbf'
lambd = 0.1
sigma = 2
model = KernelRidgeRegression(
        kernel=kernel,
        lambd=lambd,
        sigma=sigma
    ).fit(X_train, y_train)
ypred = model.predict(X_test)
print('Test error: {:.2%}'.format(error(ypred, y_test)))

Test error: 50.40%


#### Kernel Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(dtrain, y, test_size = 0.5, random_state=101)

In [689]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class KernelLogisticRegression(KernelMethodBase):
    '''
    Kernel Logistic Regression
    '''
    def __init__(self, lambd=0.1, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelLogisticRegression, self).__init__(**kwargs)

    def fit(self, X, y, max_iter=100, tol=1e-5):
    
        self.X_train = X
        self.y_train = y
        
        K = self.kernel_function_(X, X, **self.kernel_parameters)
        
        # IRLS
        WKRR = WeightedKernelRidgeRegression(
            lambd=self.lambd,
            kernel=self.kernel_name,
            **self.kernel_parameters
        )
        # Initialize
        alpha = np.zeros_like(self.y_train)
        # Iterate until convergence or max iterations
        for n_iter in range(max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(-y*f)
            alpha = WKRR.fit(K, z, sample_weights=w).alpha
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < tol:
                break
        self.n_iter = n_iter
        self.alpha = alpha

        return self
            
    def decision_function(self, X_test):
        K_x = self.kernel_function_(X_test, self.X_train, **self.kernel_parameters)
        # probability of y=1(between o and 1)
        return sigmoid(K_x.dot(self.alpha))

    def predict(self, X):
        probas =self.decision_function(X)
        predicted_classes = np.where(probas < 0.5, -1,1)
        return predicted_classes
    
    def error(self,ypred, ytrue):
        e = (ypred == ytrue).mean()
        return e

In [690]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(dtrain, y, test_size = 0.5, random_state=101)

In [691]:
kernel = 'linear'
degree = 2
sigma = 10.
lambd = 0.001
fig_title = 'Logistic Regression, {} Kernel'.format(kernel)

model = KernelLogisticRegression(lambd=lambd, kernel=kernel, sigma=sigma, degree=degree)
y_pred = model.fit(X_train, y_train).predict(X_test)
#plot_decision_function(model, X_train, y_train, title=fig_title)
print('Test error: {:.2%}'.format(error(y_pred, y_test)))

Test error: 49.60%


#### Kernel SVM

In [839]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(Data_tr, y, test_size = 0.2, random_state=42)

In [840]:
def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))

    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P =P+ eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis,:]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

#K = polynomial_kernel(Xtrain, Xtrain)
#alphas = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=1.))

In [841]:
class KernelSVM(KernelMethodBase):
    '''
    Kernel SVM Classification
    
    Methods
    ----
    fit
    predict
    '''
    def __init__(self, C=0.1, **kwargs):
        self.C = C
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelSVM, self).__init__(**kwargs)

    def fit(self, X, y, tol=1e-3):
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        self.y_train = y
        
        # Kernel matrix
        
        K = self.kernel_function_(X,X, **self.kernel_parameters)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        # Compute support vectors and bias b
        #sv = (self.alpha > 1e-3)
        sv = np.logical_and(self.alpha > tol, (self.C - self.alpha > tol))
        
        self.beta = self.alpha*y
        
        self.X_sv = X[sv]
        self.Y_sv = y[sv]

        self.bias = y[sv]-K[sv].dot(self.alpha*y)#self.X_sv.dot(X.T).dot(self.beta)
        
        self.bias = self.bias.mean()
        self.support_vector_indices = np.nonzero(sv)[0]

        return self
        
    def decision_function(self, X):
        return self.kernel_function_(X, self.X_train,**self.kernel_parameters).dot(self.beta)+self.bias
    
    def ensemble(self,X):
        prob = self.decision_function(X)
           
        return prob
        


    def predict(self, X):
        
        prob = np.sign(self.decision_function(X))
        
        predicted_classes = np.where(prob==-1, 0, 1)

        
        return predicted_classes
    
    def error(self,ypred, ytrue):
        e = (ypred == ytrue).mean()
        return e

In [842]:
kernel = 'rbf'
degree = 2
sigma = 5.
C = 10.
model = KernelSVM(C=C, kernel=kernel, sigma=sigma, degree=degree)
model.fit(X_train, y_train)
y_pred =model.predict(X_test)

print('Test error: {:.2%}'.format(error(y_pred, y_test)))

Test error: 82.00%


#### Cross validation  @kernelridgeregression

In [824]:
kernel = 'rbf'
lam = 0.001
sigma = 10.
C=10.
degree=2
y=(y+1)/2

X_cross=dtrain
y_cross=y


from sklearn.model_selection import KFold
kfold=KFold(n_splits=5)
accuracy = []
for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
    X_train, y_train = X_cross[train_index], y_cross[train_index]
    X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
    
    model_curr = KernelRidgeRegression(
        kernel=kernel,
        lambd=lam,
        sigma=sigma
    ).fit(X_train, y_train)

    y_hat = model_curr.predict(X_valid)
    accuracy = model_curr.error(y_hat,y_valid)
    
    print 'accurracy fold {i}:', {accuracy}
print 'Average accuracy is:', {np.mean(accuracy)}

accurracy fold {i}: set([0.6125])
accurracy fold {i}: set([0.6775])
accurracy fold {i}: set([0.665])
accurracy fold {i}: set([0.6575])
accurracy fold {i}: set([0.67])
Average accuracy is: set([0.67])


#### Multinomial Naive Bayes

In [823]:
y

array([1, 0, 1, ..., 1, 1, 1])

In [768]:
d_train = pd.read_csv('Xtr.csv')
d_test = pd.read_csv('Xte.csv')

In [769]:
seqq_tra = d_train['seq']
seqq_tes = d_test['seq']

In [770]:
result_tra = map(lambda x: [x[i:i+1] for i in range(0, len(x), 1)],seqq_tra )
result_tes = map(lambda x: [x[i:i+1] for i in range(0, len(x), 1)],seqq_tes )

In [771]:
df_tra = list(result_tra)
df_tes = list(result_tes)

In [772]:
dff_tra = pd.DataFrame(df_tra)
dff_tes = pd.DataFrame(df_tes)

In [773]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
hot_tra = pd.DataFrame(enc.fit_transform(dff_tra).toarray())
hot_tes = pd.DataFrame(enc.fit_transform(dff_tes).toarray())


# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer2 = CountVectorizer(analyzer='word',min_df=1, ngram_range=(2, 2))
# X2 = vectorizer2.fit_transform(hot_tra)


In [775]:
#hot_tes = pd.get_dummies(dff_tes)
hot_tes.tail()

,0,1,2,3,4,5,6,7,8,9,...,394,395,396,397,398,399,400,401,402,403
995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
997,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
998,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
999,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [811]:
class MultinomialNB:
    
    def fit(self, X, y, ls=0.01):
        self.ls = ls
        self.y_classes, y_counts = np.unique(y, return_counts=True)
        self.x_classes = [np.unique(x) for x in X.T]
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.phi_x = self.mean_X(X, y)
        self.c_x = self.count_x(X, y)
        return self
    
    def mean_X(self, X, y):
        return [[self.ls_mean_x(X, y, k, j) for j in range(len(self.x_classes))] for k in self.y_classes]
    
    def ls_mean_x(self, X, y, k, j):
        x_data = (X[:,j][y==k].reshape(-1,1) == self.x_classes[j])
        return (x_data.sum(axis=0) + self.ls ) / (len(x_data) + (len(self.x_classes) * self.ls))
    
    def get_mean_x(self, y, j):
        return 1 + self.ls / (self.c_x[y][j] + (len(self.x_classes) * self.ls))
        
    def count_x(self, X, y):
        return [[len(X[:,j][y==k].reshape(-1,1) == self.x_classes[j])
                       for j in range(len(self.x_classes))]
                      for k in self.y_classes]

    def predict(self, X):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]
    
    def compute_prob(self, x, y):
        Pxy = 1
        for j in range(len(x)):
            x_clas = self.x_classes[j]
            if x[j] in x_clas:
                i = list(x_clas).index(x[j])
                p_x_j_y = self.phi_x[y][j][i] # p(xj|y)
                Pxy *= p_x_j_y
            else:
                Pxy *= self.get_mean_x(y, j)
        return Pxy * self.phi_y[y]
    
    def evaluate(self, X, y):
        return (self.predict(X) == y).mean()

In [822]:
y

array([1, 0, 1, ..., 1, 1, 1])

In [817]:
X_cross=hot_tra
y_cross=y


from sklearn.model_selection import KFold
kfold=KFold(n_splits=5)
accuracy = []
for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
    X_train, y_train = X_cross[train_index], y_cross[train_index]
    X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]   
    
    model_curr = MultinomialNB().fit(X_train,y_train)

    y_hat = model_curr.predict(X_valid)
    accuracy = model_curr.evaluate(X_valid,y_valid)

    print ('accurracy fold {i}:', {accuracy})
print ('Average accuracy is:', {np.mean(accuracy)})

('accurracy fold {i}:', set([0.6075]))
('accurracy fold {i}:', set([0.6375]))
('accurracy fold {i}:', set([0.6125]))
('accurracy fold {i}:', set([0.6225]))
('accurracy fold {i}:', set([0.645]))
('Average accuracy is:', set([0.645]))


In [826]:
hot_tra

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [818]:
y_hat = model_curr.predict(hot_tes)

In [820]:
tes = pd.read_csv('Xte.csv')
tes['Bound'] = y_hat
subm19 = tes[['Id','Bound']]
subm19

,Id,Bound
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,1
8,8,0
9,9,1


In [821]:
subm19.to_csv('Submission19.csv',index=False)

#### Gaussian NB

In [784]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(hot_tra, y, test_size = 0.2, random_state=42)

In [785]:
import numpy as np
     
class GaussianNB:
    
    def fit(self, X, y, epsilon = 1e-10):
        self.y_classes, y_counts = np.unique(y, return_counts=True)
        self.x_classes = np.array([np.unique(x) for x in X.T])
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.u = np.array([X[y==k].mean(axis=0) for k in self.y_classes])
        self.var_x = np.array([X[y==k].var(axis=0)  + epsilon for k in self.y_classes])
        return self
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]
    
    def compute_prob(self, x, y):
        c = 1.0 /np.sqrt(2.0 * np.pi * (self.var_x[y]))
        return np.prod(c * np.exp(-1.0 * np.square(x - self.u[y]) / (2.0 * self.var_x[y])))
    
    def evaluate(self, X, y):
        return (self.predict(X) == y).mean()

In [786]:
m1 = GaussianNB()
m1.fit(X_train,y_train)
p=m1.predict(X_test)
m1.evaluate(X_test, y_test)

0.6175